# Goal of this script

Created on 01/03/2023 by Claire S

This script cleans the email data further.

It uses data outputted by the Step0_Data_Extraction script, and produces data that is used by the Step2_Assess_New_Features script.

It will:

- Split the threaded emails into singles
- Log extra features
- Export a pickle file

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import pickle

In [3]:
# pandas options
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [4]:
# import my own functions:
import sys
import ipynb
import re
sys.path.append('/project/Xelix_Project/utils')

In [5]:
from ipynb.fs.full.Regex_html_Functions import *
from ipynb.fs.full.Loop_Functions import email_splitting

## Labelled emails

In [6]:
# df = pd.read_pickle("data/Step0_Data/bank_change_emails_and_threads.pkl")
df = pd.read_pickle("data/Step0_Data/labelled_emails_and_threads.pkl")

## Unlabelled emails

In [7]:
# df = pd.read_pickle("data/Step0_Data/unlabelled_emails_and_threads.pkl")
# df = pd.read_pickle("data/Step0_Data/unlabelled_emails_and_threads_run6.pkl")

In [8]:
print(len(df))
# df.head()

14333


In [9]:
df.columns

Index(['id', 'subject', 'internal_date', 'from_address', 'to', 'bcc', 'cc',
       'reply_to', 'html_body', 'plain_text_body', 'attachments',
       'category_id', 'category_slug', 'has_attachment', 'html_body_as_text',
       'body'],
      dtype='object')

In [10]:
# df_t = df[df['id'] == 441675]

In [11]:
# i = 1
# for row in df_t.itertuples():
#     print("#" * 70, "\n\n")
#     print(f'email index: {i}')
#     i += 1
#     print(f'subject line:\t{row.subject}')
#     print(f'received from:\t{row.from_address}')
#     print(f'has attachment:\t\t{row.has_attachment}')
#     print(f'date sent:\t{row.internal_date}')
#     print(f'slug:\t\t{row.category_slug}\n')
#     print(row.body)

In [12]:
# i = 1
# for row in df[0:20].itertuples():
#     print("#" * 70, "\n\n")
#     print(f'email index: {i}')
#     i += 1
#     print(f'subject line:\t{row.subject}')
#     print(f'received from:\t{row.from_address}')
#     print(f'has attachment:\t\t{row.has_attachment}')
#     print(f'date sent:\t{row.internal_date}')
#     print(f'slug:\t\t{row.category_slug}\n')
#     print(row.body)

# Big loop to split emails, create new features

In [13]:
# from unidecode import unidecode

In [14]:
# %%time
# # grow lists iteratively, then create dataframe at the end
# iter_count_list = []
# email_id = []
# subject = []
# internal_date = []
# from_address = []
# from_name = []
# sent_to = []
# sent_to_how_many = []
# bcc = []
# cc = []
# reply_to_address = []
# attachments = []
# category_id = []
# category_slug = []
# has_attachment = []
# email_text_body = []
# initial_domain = []

# # new columns for splitting the emails
# is_part_of_thread = []
# num_in_thread = []
# thread_id = []

# # sort code related
# has_sort_code = []
# sort_code = []

# # language related
# mentions_bank = []
# mentions_bank_account = []
# mentions_change = []
# mentions_account_specifics = []

# # in the subject
# subject_mentions_bank = []
# subject_mentions_change = []

# # additional features
# mentions_statement = []
# mentions_invoice = []
# mentions_fraud = []
# has_attention_ext_flag = []
# has_alert_attach_flag = []

# # log error for normal emails too
# has_error = []

# # error emails:
# error_mail_id = []
# error_mail_body = []
# error_type = []

# ############################
# # splitting emails by this: # Disclaimer: need to check this is the same across languages etc
# split_string1 = "From: "
# split_string2 = "wrote: "

# # some regex strings I use
# find_emails_in_section = r'(?:<|\[)(?:mailto:)?(.*?)(?:>|\])'
# find_emails_in_section2 = r'(?:<|\[)(?:mailto:)?(\b\w+@\w+\.\w+\b)(?:>|\])'

# find_all_hyperlinks = r'<(.*?)>'
# # clean_slashes = r"\\|\s\\|\b\\|\s/|/|\b/|\s"
# # clean_slashes1 = r"\\|\s\\|\b\\|/|//|\s/"
# # clean_slashes = r"\b\\|\\\b"
# clean_slashes1 = r"\\(\s+)?"
# # clean_slashes2 = r"\\\b"
# # clean_slashes3 = r"\\\s|\b\\\s"

# iter_count = 0
# error_count = 0

# # change name of df here
# for line in df.itertuples():
    
#     iter_count += 1
# #     print(f'iter count {iter_count}, id: {line.id}')
    
#     ################### if single email, we just append as we would
#     if split_string1 not in line.body and split_string2 not in line.body:
        
        
#         # save thread details
#         iter_count_list.append(iter_count)
#         is_part_of_thread.append(False)
#         num_in_thread.append(False)
#         thread_id.append(False)
        
#         # save data
#         email_id.append(line.id)
#         attachments.append(line.attachments)
#         category_id.append(line.category_id)
#         category_slug.append(line.category_slug)
#         has_attachment.append(line.has_attachment)
#         first_domain = extract_domain_from_address(line.from_address)
#         initial_domain.append(first_domain)
        
#         # email specific
#         subject.append(line.subject)
#         internal_date.append(line.internal_date)
#         from_address.append(line.from_address)
#         from_name.append(line.from_address) # just save sender email instead here
#         sent_to.append(line.to)
#         sent_to_how_many.append(line.to.count(',')+1)
#         bcc.append(line.bcc)
#         cc.append(line.cc)
#         reply_to_address.append(line.reply_to)
        
# #         # remove slashes
# #         this_email = re.sub(clean_slashes1," ",line.body)
# # #         this_email = re.sub(clean_slashes2," ",line.body)
# # #         this_email = re.sub(clean_slashes3," ",line.body)
# #         this_email = this_email.replace("  "," ")
        
#         # conditionals
#         sort_code_bool, sort_code_num = find_sort_code_in_email(line.body)
#         has_sort_code.append(sort_code_bool)
#         sort_code.append(sort_code_num)
        
#         # language related
#         mentions_bank.append(find_bank_mention_in_email(line.body))
#         mentions_bank_account.append(find_bank_account_mention_in_email(line.body))
#         mentions_change.append(find_update_words_in_email(line.body))
#         mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(line.body))
        
#         # in the subject
# #         print(line.subject)
#         subject_mentions_bank.append(find_bank_mention_in_email(line.subject))
#         subject_mentions_change.append(find_update_words_in_email(line.subject))
        
#         # extra features
#         mentions_statement.append(find_statement_mention_in_email(line.body))
#         mentions_invoice.append(find_invoice_mention_in_email(line.body))
#         mentions_fraud.append(find_cybercrime_fraud_mention_in_email(line.body))
#         has_attention_ext_flag.append(find_attention_external_mention_in_email(line.body))
#         has_alert_attach_flag.append(find_alert_attach_mention_in_email(line.body))
        
#         # clean email body
#         this_email = re.sub(clean_slashes1," ",line.body)
# #         this_email = re.sub(clean_slashes2," ",line.body)
# #         this_email = re.sub(clean_slashes3," ",line.body)
#         this_email = this_email.replace("  "," ")
#         email_text_body.append(clean_text_re_links_brackets(unidecode(this_email)))
#         has_error.append(False)
        
#     ################### thread of emails
#     else:
#         if split_string1 in line.body:
#             split_emails = line.body.split(split_string1)
#         elif split_string2 in line.body:
#             split_emails = line.body.split(split_string2)
#         # try / except handling here
        
#         try:
#             for i, row in enumerate(split_emails):

#                 iter_count_list.append(iter_count)
#                 row = row.replace('\\n',' ') # replace this so we can search for strings
                
#                  # save thread details
#                 is_part_of_thread.append(True)
#                 num_in_thread.append(int(len(split_emails)-(i)))
#                 thread_id.append(str(line.id)+'_t')

#                 # save regardless of thread or not
#                 email_id.append(line.id)
#                 attachments.append(line.attachments)
#                 category_id.append(line.category_id)
#                 category_slug.append(line.category_slug)
#                 has_attachment.append(line.has_attachment)
#                 initial_domain.append(extract_domain_from_address(line.from_address))
                
#                 bcc.append(None)
#                 cc.append(None)
#                 reply_to_address.append(line.reply_to) # just keep this consistent?

#                 # for every split email which wasn't the first
#                 if i > 0:

#                     # log who sent this email in thread
#                     before_sent = row.split("Sent: ")[0]
# #                     this_email = re.findall(find_emails_in_section, before_sent)
#                     this_email = re.findall(find_emails_in_section2, before_sent) # edited
#                     row = re.sub(before_sent,"",row)

#                     if not(this_email):
#                         this_person = before_sent
#                         from_address.append(this_person) # save output
                        
#                     else:
#                         this_person = re.sub(find_all_hyperlinks,"",before_sent) 
#                         from_address.append(this_email) # save output
                        
#                     from_name.append(this_person) # save output  
                     
#                     # log when this email in thread was sent
#                     date_text, row = extract_text_between_start_end('Sent: ','To: ',None, row)
#                     internal_date.append(date_text) # save output
# #                     print(f'iter: {iter_count} email_id: {line.id}, {date_text}')

#                     # log who this email was sent to
#                     to_text, row = extract_text_between_start_end('To: ','Subject: ',None, row)
#                     to_emails = re.findall(find_all_hyperlinks, to_text)
#                     sent_to.append(to_emails) # save output
#                     sent_to_how_many.append(len(to_emails)) # save output
            
#                     # log the subject of this email
#                     this_subject, row = extract_text_between_start_end('Subject: ','      ','  ', row)
#                     subject.append(this_subject) # save output
            
#                     # additional cleaning if needed                 
#                     row = row.replace("Sent: ","").replace("Sent:","")
#                     row = row.replace("To: ","").replace("To:","")
#                     row = row.replace("Subject: ","").replace("Subject:","")
#                     row = row.replace(this_subject,"")
# #                     row = re.sub('Sent: ',"",row)
# #                     row = re.sub('To: ',"",row)
# #                     row = re.sub('Subject: ',"",row)
# #                     row = re.sub(this_subject,"",row)

#                     # conditionals
#                     sort_code_bool, sort_code_num = find_sort_code_in_email(row)
#                     has_sort_code.append(sort_code_bool)
#                     sort_code.append(sort_code_num)

#                     # language related
#                     mentions_bank.append(find_bank_mention_in_email(row))
#                     mentions_bank_account.append(find_bank_account_mention_in_email(row))
#                     mentions_change.append(find_update_words_in_email(row))
#                     mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))
                    
#                     # in the subject
#                     subject_mentions_bank.append(find_bank_mention_in_email(this_subject))
#                     subject_mentions_change.append(find_update_words_in_email(this_subject))
                    
#                     # extra features
#                     mentions_statement.append(find_statement_mention_in_email(row))
#                     mentions_invoice.append(find_invoice_mention_in_email(row))
#                     mentions_fraud.append(find_cybercrime_fraud_mention_in_email(row))
#                     has_attention_ext_flag.append(find_attention_external_mention_in_email(row))
#                     has_alert_attach_flag.append(find_alert_attach_mention_in_email(row))

#                     # remove slashes
#                     row = re.sub(clean_slashes1," ",row)
# #                     row = re.sub(clean_slashes2," ",row)
# #                     row = re.sub(clean_slashes3," ",row)
#                     row = row.replace("  "," ")
    
#                     # clean email body
#                     email_text_body.append(clean_text_re_links_brackets(unidecode(row)))
#                     has_error.append(False) # ran without error
                
#                 elif i == 0:
                    
#                     # first email in thread, save as if saving from the main data
#                     subject.append(line.subject)
#                     internal_date.append(line.internal_date)
#                     from_address.append(line.from_address)
#                     from_name.append(line.from_address) # just save sender email instead here
#                     sent_to.append(line.to)
#                     sent_to_how_many.append(line.to.count(',')+1)

#                     # conditionals
#                     sort_code_bool, sort_code_num = find_sort_code_in_email(row)
#                     has_sort_code.append(sort_code_bool)
#                     sort_code.append(sort_code_num)

#                     # language related
#                     mentions_bank.append(find_bank_mention_in_email(row))
#                     mentions_bank_account.append(find_bank_account_mention_in_email(row))
#                     mentions_change.append(find_update_words_in_email(row))
#                     mentions_account_specifics.append(find_bank_vocabulary_mention_in_email(row))
                    
#                     # in the subject
#                     subject_mentions_bank.append(find_bank_mention_in_email(line.subject))
#                     subject_mentions_change.append(find_update_words_in_email(line.subject))
                    
#                     # extra features
#                     mentions_statement.append(find_statement_mention_in_email(row))
#                     mentions_invoice.append(find_invoice_mention_in_email(row))
#                     mentions_fraud.append(find_cybercrime_fraud_mention_in_email(row))
#                     has_attention_ext_flag.append(find_attention_external_mention_in_email(row))
#                     has_alert_attach_flag.append(find_alert_attach_mention_in_email(row))

#                     # remove slashes
#                     row = re.sub(clean_slashes1," ",row)
# #                     row = re.sub(clean_slashes2," ",row)
# #                     row = re.sub(clean_slashes3," ",row)
#                     row = row.replace("  "," ")
                    
#                     # clean email body
#                     email_text_body.append(clean_text_re_links_brackets(unidecode(row)))
#                     has_error.append(False) # ran without error
                    
#         # email threw an error
#         except re.error as e:
            
#             error_count += 1
#             has_error.append(True) # ran with error
            
#             # need to count if all lists are same length - error could have happened somewhere
#             arrays = [iter_count_list, email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
#                      bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
#                      email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
#                      mentions_bank,mentions_bank_account,mentions_change,mentions_account_specifics,has_error,
#                      mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,
#                      has_alert_attach_flag,initial_domain,subject_mentions_bank,subject_mentions_change]
            
#             # add None's to emails with an error - this could probably be smarter
#             [iter_count_list,email_id, subject, internal_date, from_address, from_name, sent_to,sent_to_how_many,
#              bcc,cc,reply_to_address,attachments,category_id,category_slug,has_attachment,
#              email_text_body, is_part_of_thread, num_in_thread, thread_id, has_sort_code,sort_code,
#              mentions_bank,mentions_bank_account,mentions_change,mentions_account_specifics,has_error,
#              mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,
#                      has_alert_attach_flag,initial_domain,subject_mentions_bank,subject_mentions_change] = check_array_lengths_add_Nones(arrays)
            
#             # log details for error emails
#             error_mail_id.append(line.id)
#             error_mail_body.append(line.body) 
#             error_type.append(e)
            
# #             ################## if wanted ####################
# #             # print out the error emails only
# #             print("#" * 70, "\n\n")
# #             print(f'Regex error occurred: {e}\n\n')               
# #             print(f'email id: {line.id}, iter number: {iter_count}. So far there have been {error_count} errors.\n\n')
# #             print(f'subject line:\t{line.subject}')
# #             print(f'received from:\t{line.from_address}')
# #             print(f'sent to:\t{line.to}')
# #             #print(f'emails sent to: {line.to.count(',')+1} people')
# #             print(f'date sent:\t{line.internal_date}')
# #             print(f'slug:\t\t{line.category_slug}')
# #             print("")
# #             print(line.body)
# #             print("\n")
# #             ##################################################

# print("-" * 50, "\n")
# print(f'there were {error_count} errors in total.\n')
# print("-" * 50)    

In [15]:
%%time
new_df, error_df = email_splitting(df)

-------------------------------------------------- 

there were 7 errors in total.

--------------------------------------------------
CPU times: user 9.5 s, sys: 25.1 ms, total: 9.53 s
Wall time: 9.53 s


In [16]:
print(len(error_df))
error_df.head(10)

7


,email_id,email_text_body,error_type
0,195490,Alert: There is currently a serious threat fro...,bad character range o-g at position 40 (line 1...
1,287962,Attention: This email originated outside of ou...,bad character range o-g at position 34 (line 1...
2,287962,Attention: This email originated outside of ou...,bad character range o-g at position 34 (line 1...
3,288058,Attention: This email originated outside of ou...,bad character range o-g at position 42 (line 1...
4,289288,Attention: This email originated outside of...,bad character range o-g at position 36
5,289288,Attention: This email originated outside of...,bad character range o-g at position 36
6,307428,Hi Stuart I will chase up on your month...,bad character range o-g at position 33


In [17]:
print(len(new_df))
new_df.head(40)

17963


,iter_count_list,email_id,is_part_of_thread,num_in_thread,thread_id,initial_domain,from_address,from_name,sent_to,sent_to_how_many,bcc,cc,reply_to_address,internal_date,category_id,category_slug,has_sort_code,sort_code,has_attachment,mentions_bank,mentions_change,mentions_account_specifics,mentions_bank_account,subject_mentions_bank,subject_mentions_change,attachment_mentions_bank,mentions_statement,mentions_invoice,mentions_fraud,has_attention_ext_flag,has_alert_attach_flag,subject,attachments,email_text_body
0,1,167529,False,0,False,bluestemgroup.co.uk,sales@bluestemgroup.co.uk,sales@bluestemgroup.co.uk,{Purchase.Ledger@evo-group.co.uk},1,{},{},{},2021-11-17 16:08:00+00,100,invoices,False,None,False,False,False,False,False,False,False,False,False,False,False,True,False,BLUESTEM GROUP INV PP2422423 - Order X843348. ...,None,Attention: This email originated outside of ...
1,2,167544,False,0,False,albion.co.uk,sefika@albion.co.uk,sefika@albion.co.uk,{Matt.Hales@evo-group.co.uk},1,{},{Ryan.Leigh@evo-group.co.uk},{},2022-07-05 15:37:48+00,1,statement,False,None,True,False,False,False,False,False,False,False,True,False,False,True,False,"Overdue Account £59,632.58",Followups(6).pdf,Attention: This email originated outside of ...
2,3,170741,True,10,170741_t,evo-group.co.uk,Lisa.Dyson@evo-group.co.uk,Lisa.Dyson@evo-group.co.uk,{Purchase.Ledger@evo-group.co.uk},1,None,None,{},2021-07-28 10:10:32+00,100,invoices,False,None,False,False,False,False,False,False,False,False,False,False,False,False,False,FW: APRIL ACCOUNT,None,Lisa Dyson Group Accounts Payable Supervi...
3,3,170741,True,9,170741_t,evo-group.co.uk,Peter Bown <peter.bown@slingsby.com>,Peter Bown <peter.bown@slingsby.com>,[Lisa.Dyson@evo-group.co.uk],1,None,None,{},28 July 2021 10:53,100,invoices,False,None,False,False,False,False,False,False,False,None,False,True,False,True,False,RE: APRIL ACCOUNT,None,Attention: This email originated outside of...
4,3,170741,True,8,170741_t,evo-group.co.uk,Lisa Dyson < Lisa.Dyson@evo-group.co.uk >,Lisa Dyson < Lisa.Dyson@evo-group.co.uk >,[ peter.bown@slingsby.com ],1,None,None,{},28 July 2021 10:35,100,invoices,False,None,False,False,False,False,False,False,False,None,False,False,False,False,False,RE: APRIL ACCOUNT,None,Thanks Peter Could you send me the credit...
5,3,170741,True,7,170741_t,evo-group.co.uk,Peter Bown < peter.bown@slingsby.com >,Peter Bown < peter.bown@slingsby.com >,[ Lisa.Dyson@evo-group.co.uk ],1,None,None,{},28 July 2021 10:31,100,invoices,False,None,False,False,False,False,False,False,False,None,True,False,False,True,False,RE: APRIL ACCOUNT,None,Attention: This email originated outside of...
6,3,170741,True,6,170741_t,evo-group.co.uk,Lisa Dyson < Lisa.Dyson@evo-group.co.uk >,Lisa Dyson < Lisa.Dyson@evo-group.co.uk >,[ peter.bown@slingsby.com ],1,None,None,{},28 July 2021 09:49,100,invoices,False,None,False,False,False,False,False,False,False,None,False,False,False,False,False,RE: APRIL ACCOUNT,None,"HI Pete Is our account on hold, if so is i..."
7,3,170741,True,5,170741_t,evo-group.co.uk,Peter Bown < peter.bown@slingsby.com >,Peter Bown < peter.bown@slingsby.com >,"[ Lisa.Dyson@evo-group.co.uk , Matt.Hales@evo...",4,None,None,{},15 July 2021 14:43,100,invoices,False,None,False,False,True,False,False,False,False,None,True,False,False,True,False,FW: APRIL ACCOUNT,None,Attention: This email originated outside of...
8,3,170741,True,4,170741_t,evo-group.co.uk,Matt Hales < Matt.Hales@evo-group.co.uk >,Matt Hales < Matt.Hales@evo-group.co.uk >,"[ joanne.mcewan@slingsby.com , Lisa.Dyson@evo...",4,None,None,{},15 July 2021 14:19,100,invoices,False,None,False,False,False,False,False,False,False,None,False,False,False,False,False,RE: APRIL ACCOUNT,None,Hi Joanne Will look into as soon as I can;...
9,3,170741,True,3,170741_t,evo-group.co.uk,Joanne McEwan < joanne.mcewan@slingsby.com >,Joanne McEwan < joanne.mcewan@slingsby.com >,"[ Matt.Hales@evo-group.co.uk , morgan.morris@...",3,None,None,{},13 July 2021 14:08,100,invoices,False,

In [19]:
print(pd.unique(new_df['attachment_mentions_bank']))
print(pd.unique(new_df['subject_mentions_bank']))

[False None True]
[False True None]


# Export to Pickle

In [20]:
new_df.to_pickle("data/Step1_Data/labelled_emails_Step1_Split_newfeatures_V2.pkl")
error_df.to_pickle("data/Step1_Data/labelled_emails_Step1_error_emails_V2.pkl")

### Combine all lists into a dictionary

In [ ]:
dict_data = {'iter_count_list':iter_count_list,'email_id':email_id,'is_part_of_thread':is_part_of_thread,'num_in_thread':num_in_thread,'thread_id':thread_id,
             'initial_domain':initial_domain,'from_address':from_address,'from_name':from_name,'sent_to':sent_to,
             'sent_to_how_many':sent_to_how_many,
             'bcc':bcc,'cc':cc,'reply_to_address':reply_to_address,
             'internal_date':internal_date,'category_id':category_id,'category_slug':category_slug,
             'has_sort_code':has_sort_code,'sort_code':sort_code,'has_attachment':has_attachment,
             'mentions_bank':mentions_bank,'mentions_change':mentions_change,'mentions_account_specifics':mentions_account_specifics, 
             'mentions_statement':mentions_statement,'mentions_invoice':mentions_invoice,'mentions_fraud':mentions_fraud,
             'has_attention_ext_flag':has_attention_ext_flag,'has_alert_attach_flag':has_alert_attach_flag,
             'subject':subject,'attachments':attachments,'email_text_body':email_text_body
            }

# check dictionary items are same length
check_dict_value_lengths(dict_data)

In [ ]:
error_dict_data = {'email_id':error_mail_id,'email_text_body':error_mail_body,'error_type':error_type}
# check dictionary items are same length
check_dict_value_lengths(error_dict_data)

In [ ]:
new_df = pd.DataFrame.from_dict(dict_data)
error_df = pd.DataFrame.from_dict(error_dict_data)

In [ ]:
print(len(error_df))
error_df.head()

Line below turns numeric columns into integers, replaces missing with 0

In [ ]:
new_df[['iter_count_list','email_id','num_in_thread','category_id','sent_to_how_many']] = new_df[['iter_count_list','email_id','num_in_thread','category_id','sent_to_how_many']].fillna(0).astype(int)

In [ ]:
print(len(new_df))
new_df.head(40)

## If want to read some

In [ ]:
# i = 1
# for row in new_df[0:50].itertuples():
#     print("#" * 70, "\n\n")
#     print(f'email index: {i}')
#     i += 1
#     print(f'subject line:\t{row.subject}')
#     print(f'received from:\t{row.from_address}')
#     print(f'is part of thread:\t{bool(row.is_part_of_thread)}')
#     #print(f'num in thread:\t\t{int(row.num_in_thread)}')
#     print(f'has attachment:\t\t{row.has_attachment}')
#     print(f'has sort code:\t\t{row.has_sort_code}')
#     print(f'mentions bank:\t\t{row.mentions_bank}')
#     print(f'mentions b details:\t{row.mentions_account_specifics}')
#     print(f'mentions change:\t{row.mentions_change}')
#     #print(f'sent to:\t{row.sent_to}')
#     print(f'sent to {row.sent_to_how_many} people')
#     print(f'date sent:\t{row.internal_date}')
#     print(f'slug:\t\t{row.category_slug}\n')
#     print(row.email_text_body)

# Export split files as pickle files again

In [ ]:
# new_df.to_pickle("data/Step1_Data/bank_change_emails_Step1_Split_newfeatures.pkl")

In [ ]:
# new_df.to_pickle("data/Step1_Data/labelled_emails_Step1_Split_newfeatures.pkl")
# error_df.to_pickle("data/Step1_Data/labelled_emails_Step1_error_emails.pkl")

In [ ]:
new_df.to_pickle("data/Step1_Data/unlabelled_emails_Step1_Split_newfeatures_run6.pkl")
error_df.to_pickle("data/Step1_Data/unlabelled_emails_Step1_error_emails_run6.pkl")

In [ ]:
del new_df, error_df